# Лабораторная работа №1

## Библиотеки
В ходе всех лабораторных работ будут использоваться следующие библиотеки:

Библиотека|Описание
-|-
[numpy](http://www.numpy.org/)|Работа с массивами и матрицами
[pandas](http://pandas.pydata.org/)|Удобная обработка наборов данных
[scikit-learn](https://scikit-learn.org/stable/)|Алгоритмы и методы машинного обучения
 
Если для выполнения какой-либо из лабораторных работ потребуются дополнительные библиотеки, об этом будет сказано в начале тетрадки соответствующей лабораторной работы.

## Задание
Лабораторная работа состоит из нескольких этапов:
 1. Импортировать данные из файла
 2. Предобработать данные (нормализация, создание и уменьшение числа признаков)
 3. Создать простейшую линейную модель и обучить её на наборе данных
 4. Определить точность данной модели
 5. Понять важность правильного отбора признаков, на которые обращает внимание модель

Ниже представлена минимальная теоретическая информация, необходимая для выполнения задания, куски кода для примера, а также более детальное пояснение каждого из этапов.

## Инструкция
### Этап 1: Импорт данных
Для импорта данных удобно использовать библиотеку pandas.  
Чтобы иметь возможность использовать pandas, необходимо выполнить:  

In [2]:
import pandas as pd

Теперь можно импортировать данные. Так как все наборы данных для данной лабы представлены в формате CSV, можем смело пользоваться методом pandas для считывания данных в формате CSV:  

In [16]:
df = pd.read_csv('datasets/titanic.csv')

Метод `read_csv()` принимает в качестве аргумента имя файла и кучу других параметров. Подробнее о методе `read_csv()` можно прочитать [здесь](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html).  
Но что же возвращает этот метод? Он возвращает содержимое файла, упакованное в структуру данных `DataFrame`, которая является  одной из основных в библиотеке pandas.

DataFrame -- двумерная структура данных, с размеченными столбцами и строками. По сути, просто таблица с удобным интерфейсом доступа к данным. Ниже перечислены несколько удобных способов доступа к данным:  

Способ доступа|Описание|Пример
-|-|-
`df.head([n])`|Получить первые n строк|`df.head(5)`
`df[str]`|Выбрать столбец|`df['sex']`
`df.iloc[idx]`|Выбрать строку(и)|`df.iloc[2:6]`
`df.at[idx, str]`|Выбрать ячейку|`df.at[4, 'sex']`

Изменять данные можно обычным оператором присваивания (=).  
Некоторые ресурсы для дальнейшего ознакомления:
 * [Легкий для чтения гайд по структурам данных в pandas](https://pandas.pydata.org/pandas-docs/stable/getting_started/dsintro.html#dataframe)
 * [Чуть подробнее про разметку и доступ к данным](https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#indexing)
 * [Документация по DataFrame](https://pandas.pydata.org/pandas-docs/stable/reference/frame.html)

### Этап 2: Предобработка данных
Теперь, когда данные загружены в память, можно приступать к их предобработке. 
Сначала, посмотрим, что же вообще за данные там лежат:

In [17]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


Видим, что данные в столбце "пол", не унифицированы. То бишь, один и тот же пол может обозоначаться как "male", "Male", "MALE" и т. д.  
Такая ситуация крайне нежелательна, и от таких ситуаций необходимо избавляться.  
Кроме того, модель на вход принимает только числовые, таким образом каждому полу нужно сопоставить некоторое целое число, и преобразовать столбец пола нужным образом.  
Более подробно о том, что именно необходимо сделать на данном этапе, см. ниже в таблице вариантов.

In [18]:
def normalize_sex(df: pd.DataFrame) -> pd.DataFrame:
    df['Sex'] = df['Sex'].apply(lambda x: 0 if x == 'male' else 1 if x == 'female' else 2)
    # Implement this
    return df

df = normalize_sex(df)
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",0,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",1,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",1,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",1,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",0,35.0,0,0,373450,8.0500,NaN,S


### Этап 3: Создание и обучение линейной модели

In [19]:
temp = df

Загрузить датасет в формате CSV согласно варианту.
Произвести согласно варианту:
 * Унификацию данных
 * Приведение отношений к первой нормальной форме
 * Создание признаков
 * Получение информации из данных
 * Отбор признаков с помощью:
     1. Случайного отбора(3 комбинации), сравнить с заданным контрольным.
     2. Регуляризации по данной конфигурации нейросетевой модели.
 * Произвести кросс-валидацию с разделением на N тестовых наборов и 1 контрольный
 * Оценить качество результата с помощью:
     1. Метрик(presicion, recall, F-measure)
     2. Функции потерь
     3. Q-функции
 * Построить линейную модель
 
 Каждый этап форматировать по образцу🤔)

### Образец заголовка

In [8]:
🤔)#Образец кода

SyntaxError: invalid character in identifier (<ipython-input-8-b707d67171e9>, line 1)

## Варианты

| Вариант| Датасет   | Создаваемые признаки/Унификация | N   | Результат |
|-|-|-|-|-|
| 1 | heart_diseases.csv | Унификация | 3 | Зоны риска сердечных заболеваний |
| 2 | mushroom.csv | Унификация | 5 | Съедобность гриба |
| 3 | houses.csv | Унификация | 4 | Стоимость дома |
| 4 | alcohol.csv | Унификация | 4 | Склонность к употреблению спиртного  |
| 5 | sales.csv | Унификация | 5 | Размер зарплаты |
| 6 | wines.csv | Классифицировать содержание сахара(сухое, полусухое, ...) | 3 | Вкусовые качества вина |
| 7 | passengers.csv | Унификация | 3 | Выжил ли пассажир |


# дрон дальше не зырь

### Импорт зависимостей

In [4]:
import pandas as pd

In [5]:
df = pd.read_csv('wine reviews.csv')

In [9]:
df.head()

,id,asins,brand,categories,dateAdded,dateUpdated,descriptions,dimension,ean,flavors,...,reviews.sourceURLs,reviews.text,reviews.title,reviews.userCity,reviews.userProvince,reviews.username,sizes,sourceURLs,upc,weight
0,AV13ClKCGV-KLJ3akN68,NaN,Gallo,"Food & Beverage,Beverages,Wine, Beer & Liquor,...",2017-07-24T23:59:11Z,2018-01-10T18:06:28Z,NaN,1.0 in x 1.0 in x 1.0 in,NaN,NaN,...,https://redsky.target.com/groot-domain-api/v1/...,This a fantastic white wine for any occasion!,My Favorite White Wine,NaN,NaN,Bjh,NaN,http://redsky.target.com/v1/plp/search?kwr=y&c...,4.9213E+11,1.0 lbs
1,AV13CsvW-jtxr-f38AQO,NaN,Fresh Craft Co.,"Food & Beverage,Beverages,Wine, Beer & Liquor,...",2017-07-24T23:59:42Z,2018-01-10T05:38:33Z,"[{""dateSeen"":[""2017-12-21T05:43:00.000Z"",""2017...",4.25 in x 4.25 in x 5.25 in,NaN,NaN,...,https://redsky.target.com/groot-domain-api/v1/...,"Tart, not sweet...very refreshing and delicious!",Yum!!,NaN,NaN,Wino,NaN,http://redsky.target.com/v1/plp/search?kwr=y&c...,83120003441,2.45 lbs
2,AV13CVI_glJLPUi8O7Po,NaN,1000 Stories,"Food & Beverage,Beverages,Wine, Beer & Liquor,...",2017-07-24T23:58:05Z,2018-01-10T05:38:31Z,NaN,3.3 in x 3.3 in x 11.79 in,NaN,NaN,...,https://redsky.target.com/groot-domain-api/v1/...,I was given this wine so it was a delightful s...,A New Favorite!,NaN,NaN,Bama Mom,NaN,http://redsky.target.com/v1/plp/search?kwr=y&c...,82896001453,3.09 lbs
3,AV13CVI_glJLPUi8O7Po,NaN,1000 Stories,"Food & Beverage,Beverages,Wine, Beer & Liquor,...",2017-07-24T23:58:05Z,2018-01-10T05:38:31Z,NaN,3.3 in x 3.3 in x 11.79 in,NaN,NaN,...,https://redsky.target.com/groot-domain-api/v1/...,This is a phenomenal wine and my new favorite ...,"Bold, Flavorful, Aromatic, Delicious",NaN,NaN,Av Dub,NaN,http://redsky.target.com/v1/plp/search?kwr=y&c...,82896001453,3.09 lbs
4,AV13CYL4-jtxr-f37_-t,NaN,Wine Cube153,"Food & Beverage,Beverages,Wine, Beer & Liquor,...",2017-07-24T23:58:18Z,2018-01-10T18:06:29Z,"[{""dateSeen"":[""2017-12-21T05:43:00.000Z"",""2017...",1.0 in x 1.0 in x 1.0 in,NaN,NaN,...,https://redsky.target.com/groot-domain-api/v1/...,4 750ml bottles for the price of two With way ...,"Yum! Plus, Environmentally Friendly!",Overland Park,NaN,Chelseamay,NaN,https://redsky.target.com/groot-domain-api/v1/...,85200600465,1.0 lbs


In [10]:
changed = df.replace({'brand': 'Gallo'}, '🤔)')

In [11]:
changed.head()

,id,asins,brand,categories,dateAdded,dateUpdated,descriptions,dimension,ean,flavors,...,reviews.sourceURLs,reviews.text,reviews.title,reviews.userCity,reviews.userProvince,reviews.username,sizes,sourceURLs,upc,weight
0,AV13ClKCGV-KLJ3akN68,NaN,🤔),"Food & Beverage,Beverages,Wine, Beer & Liquor,...",2017-07-24T23:59:11Z,2018-01-10T18:06:28Z,NaN,1.0 in x 1.0 in x 1.0 in,NaN,NaN,...,https://redsky.target.com/groot-domain-api/v1/...,This a fantastic white wine for any occasion!,My Favorite White Wine,NaN,NaN,Bjh,NaN,http://redsky.target.com/v1/plp/search?kwr=y&c...,4.9213E+11,1.0 lbs
1,AV13CsvW-jtxr-f38AQO,NaN,Fresh Craft Co.,"Food & Beverage,Beverages,Wine, Beer & Liquor,...",2017-07-24T23:59:42Z,2018-01-10T05:38:33Z,"[{""dateSeen"":[""2017-12-21T05:43:00.000Z"",""2017...",4.25 in x 4.25 in x 5.25 in,NaN,NaN,...,https://redsky.target.com/groot-domain-api/v1/...,"Tart, not sweet...very refreshing and delicious!",Yum!!,NaN,NaN,Wino,NaN,http://redsky.target.com/v1/plp/search?kwr=y&c...,83120003441,2.45 lbs
2,AV13CVI_glJLPUi8O7Po,NaN,1000 Stories,"Food & Beverage,Beverages,Wine, Beer & Liquor,...",2017-07-24T23:58:05Z,2018-01-10T05:38:31Z,NaN,3.3 in x 3.3 in x 11.79 in,NaN,NaN,...,https://redsky.target.com/groot-domain-api/v1/...,I was given this wine so it was a delightful s...,A New Favorite!,NaN,NaN,Bama Mom,NaN,http://redsky.target.com/v1/plp/search?kwr=y&c...,82896001453,3.09 lbs
3,AV13CVI_glJLPUi8O7Po,NaN,1000 Stories,"Food & Beverage,Beverages,Wine, Beer & Liquor,...",2017-07-24T23:58:05Z,2018-01-10T05:38:31Z,NaN,3.3 in x 3.3 in x 11.79 in,NaN,NaN,...,https://redsky.target.com/groot-domain-api/v1/...,This is a phenomenal wine and my new favorite ...,"Bold, Flavorful, Aromatic, Delicious",NaN,NaN,Av Dub,NaN,http://redsky.target.com/v1/plp/search?kwr=y&c...,82896001453,3.09 lbs
4,AV13CYL4-jtxr-f37_-t,NaN,Wine Cube153,"Food & Beverage,Beverages,Wine, Beer & Liquor,...",2017-07-24T23:58:18Z,2018-01-10T18:06:29Z,"[{""dateSeen"":[""2017-12-21T05:43:00.000Z"",""2017...",1.0 in x 1.0 in x 1.0 in,NaN,NaN,...,https://redsky.target.com/groot-domain-api/v1/...,4 750ml bottles for the price of two With way ...,"Yum! Plus, Environmentally Friendly!",Overland Park,NaN,Chelseamay,NaN,https://redsky.target.com/groot-domain-api/v1/...,85200600465,1.0 lbs
